In [3]:
!pip install surprise

     |████████████████████████████████| 11.8MB 8.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617579 sha256=feb6cfbca24ad3e6bfb4d661a10984e8d9949844fdda323154e5c63c860085b3
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
from surprise import Dataset
from surprise import Reader

from collections import defaultdict
import numpy as np
import pandas as pd
import io
import re


# Menupan Crawling Data Load

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/menupan_crawling.csv')
data.head()

,Unnamed: 0,res_name,res_type,res_address,res_homepage,res_score,res_theme,res_hour,res_room,smoking,amenity,parking,res_info,review,rev_date,rev_score
0,0,레빠쥬(les pages),양식,서울 강남구 신사동 506 강남출판문화센터 1층,http://www.lespages.co.kr/,3.8,"술 못마시는 친구와 함께, 일상데이트, 점심식사, 야외테라스, 스테이크, 와인, 양...",11:30 ~ 22:00BreakTime: 15:30~17:00,50석 / 방: 방없음실내 36석 / 테라스 16석,모두 금연석,Wifi,"4대, 주차방법: 무료, 주차장소: 업소앞/건물 지하주차장 (1시간 무료이용)","가로수길 뒷편에 자리잡은 피자, 파스타, 샌드위치를 판매하는 이탈리안 가정식 레스토...",파스타가 괜찮은 음식점@@\n감사합니다. 추천할께요^^@@\n관자의 쫄길함이 살아있...,2014/04/09\n2013/12/11\n2015/07/27\n2015/03/17...,3.0\n5.0\n3.0\n4.0\n5.0\n3.0\n3.0\n3.0\n5.0\n4.0
1,1,듀크렘(DEUX CREMES),카페/주점-카페,서울 강남구 신사동 545-4,Nan,3.9,"술 못마시는 친구와 함께, 외국 현지 음식(for 외국인), 소개팅, 일상데이트, ...",11:00 ~ 23:00,100석 / 방: 방없음,모두 금연석,Wifi,"100대, 주차방법: 유료, 주차장소: 발렛파킹/발렛비 : 2,000원",가로수길에서 타르트 맛집으로 이미 핫플레이스인 듀크렘.말 그대로 두가지 크림이라는 ...,직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들@@\n타르트...,2014/04/02\n2014/04/25\n2015/04/11\n2014/04/30...,1.3\n3.5\n4.0\n4.5\n4.5\n3.0\n2.5\n1.3\n4.0\n4...
2,2,팬아시아 신사가로수길점,기타/세계-동남아음식,서울 강남구 신사동 546-23 2층,Nan,4.1,"가족외식, 술 마시는 친구와 함께, 일상데이트, 브런치, 와인, 아시안 면요리, 타...",12:00 ~ 23:00,72석 / 방: 방없음,모두 금연석,Wifi,주차안됨,'잊을수 없는 경험' 이라는 신조가 팬아시아 레스토랑의 고객에 대한 마음가짐이다.팬...,보기에도 한번 가보고 싶은 곳이네요 ???@@\n동남아음식도 좋네요''''ㅎㅎ@@\...,2018/10/25\n2015/03/31\n2014/07/14\n2014/06/11...,4.5\n4.0\n5.0\n3.0\n4.5\n5.0\n3.5\n4.0\n5.0\n4...
3,3,더가든키친(The Garden Kitchen),호텔뷔페,서울 서초구 잠원동 6-1 더리버사이드호텔 1층,http://www.riversidehotel.co.kr,4.3,"가족외식, 돌잔치/회갑/생일, 어른과 함께라면, 아이와 함께라면, 일상데이트, 특별...",11:30 ~ 21:30,600석 / 방: 13개,모두 금연석,"놀이방,Wifi","200대, 주차방법: 유료, 주차장소: 발렛파킹","<리버사이드 더가든키친 할인이벤트>주중점심 53,200 ->39,900(25%)주중...",기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요^0^...,2015/10/01\n2017/02/13\n2016/10/11\n2015/12/16...,4.3\n4.5\n4.8\n4.0\n4.3\n4.3\n3.8\n4.0\n4.8\n4.5
4,4,시리얼고메(Serial Gourmet),양식,서울 강남구 신사동 541-13 2층,http://blog.naver.com/gourmet01,3.4,"소개팅, 특별한 날엔, 브런치, 주차공간, 스테이크, 맥주, 와인, 칵테일, 양식 ...",11:00 ~ 22:00일 11시~21시30분 / 브레이크 타임 없음,60석,모두 금연석,Nan,"10대, 주차방법: 유료, 주차장소: 발렛파킹/3,000원(2시간)","스타셰프 맛집 : 레이먼킴 쉐프 (오늘뭐먹지, 올리브쇼)시리얼고메는 유명 쉐프 레이...",맛도 그닥...서비스는 최악이더군요@@\n친구랑 같이 브런치 먹었는데 분위기도 좋고...,2016/04/02\n2015/12/18\n2016/03/14\n2015/04/08...,1.0\n4.0\n1.0\n4.0\n5.0\n5.0\n3.0\n4.0\n3.0\n4.3


# Preprocessing

In [27]:
menupan = data[['res_name','review']]
menupan.head()

,res_name,review
0,레빠쥬(les pages),파스타가 괜찮은 음식점@@\n감사합니다. 추천할께요^^@@\n관자의 쫄길함이 살아있...
1,듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들@@\n타르트...
2,팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 ???@@\n동남아음식도 좋네요''''ㅎㅎ@@\...
3,더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요^0^...
4,시리얼고메(Serial Gourmet),맛도 그닥...서비스는 최악이더군요@@\n친구랑 같이 브런치 먹었는데 분위기도 좋고...


In [35]:
## 이모티콘 제거 compile 
## 제거 안되는 이모티콘이 있음!!!!!!!!!!!!!!!!
def rmEmoji(inputData):
    text = inputData.encode('utf-8', 'ignore').decode('utf-8')
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

print(rmEmoji('만들어주세요❤️'))

만들어주세요❤


In [31]:
## 한글만 추출
re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+').findall('맛있게 만들어주세요❤️')

['맛있게', '만들어주세요']

In [32]:
menupan.loc[3965,'review']

"안가본사람은 있어도 한번만 가본사람은 없는곳. 이탈리아 여행길에서 먹었던 파씨보다 쌀 젤라또가 맛있다.@@\n입에 살살 녹는 맛~~ 크으~@@\n아포가또 호두가 맛있어요@@\n아포가또가 먹고싶어요@@\n쫀득한 맛이 일품이예요!@@\n맛있는 젤라또인건 분명한데..좀 비싼감도 있고 앉을자리가 없어서 좀 불편한정도..@@\n감사합니다. 추천합니다.@@\n이천쌀은 항상 필수!! 모든 메뉴 맛있기에 다른건 돌아가면서 먹어요..^^@@\n이천쌀맛 최고예요 잠실에도 만들어주세요❤️ (장미상가는 상가세가 상대적으로 괜찮은데 요새 삼성회사가 들어오면서 장사는 엄청 잘된대요)친구들도 원하고요~ 생기면 맨날 갈거예요 !!! ❤@@\n여ㅛ친과 가기 좋은곳''''ㅎㅎ@@\n여기 여자친구 데려가면 센스있다고 칭찬받을수 있어요...ㅋㅋㅋㅋ@@\n"

In [44]:
k = 0

for i in menupan['review']:
  text = re.compile('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+').findall(i)
  menupan.loc[k, 'review'] = ' '.join(text).strip()
  k+=1

In [45]:
menupan.loc[3965,'review']

'안가본사람은 있어도 한번만 가본사람은 없는곳 이탈리아 여행길에서 먹었던 파씨보다 쌀 젤라또가 맛있다 입에 살살 녹는 맛 크으 아포가또 호두가 맛있어요 아포가또가 먹고싶어요 쫀득한 맛이 일품이예요 맛있는 젤라또인건 분명한데 좀 비싼감도 있고 앉을자리가 없어서 좀 불편한정도 감사합니다 추천합니다 이천쌀은 항상 필수 모든 메뉴 맛있기에 다른건 돌아가면서 먹어요 이천쌀맛 최고예요 잠실에도 만들어주세요 장미상가는 상가세가 상대적으로 괜찮은데 요새 삼성회사가 들어오면서 장사는 엄청 잘된대요 친구들도 원하고요 생기면 맨날 갈거예요 여ㅛ친과 가기 좋은곳 ㅎㅎ 여기 여자친구 데려가면 센스있다고 칭찬받을수 있어요 ㅋㅋㅋㅋ'

In [46]:
menupan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4177 entries, 0 to 4176
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   res_name  4177 non-null   object
 1   review    4177 non-null   object
dtypes: object(2)
memory usage: 257.9+ KB


In [47]:
## review 특수문자 제거
menupan['review'] = menupan['review'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
menupan.head()

,res_name,review
0,레빠쥬(les pages),파스타가 괜찮은 음식점 감사합니다 추천할께요 관자의 쫄길함이 살아있다 이곳 파스타가...
1,듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들 타르트가 촉...
2,팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 동남아음식도 좋네요 ㅎㅎ 동남아음식에 대한 거부...
3,더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요 맛있...
4,시리얼고메(Serial Gourmet),맛도 그닥 서비스는 최악이더군요 친구랑 같이 브런치 먹었는데 분위기도 좋고 맛있었어...


In [50]:
## 리뷰없는 식당 삭제
print(len(menupan))
drop_index = menupan[menupan['review'] == ''].index
menupan = menupan.drop(drop_index)
print(len(menupan))

4119
4119


In [9]:
# ## 가게 1000개만 추려서 저!장!
# menupan1000 = menupan.loc[:999,:]
# print(menupan1000.shape)
# menupan1000.head()

(10, 2)


,res_name,review
0,레빠쥬(les pages),파스타가 괜찮은 음식점 감사합니다 추천할께요 관자의 쫄길함이 살아있다 ...
1,듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들 타르트가...
2,팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 동남아음식도 좋네요 ㅎㅎ 동...
3,더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요 0 ...
4,시리얼고메(Serial Gourmet),맛도 그닥 서비스는 최악이더군요 친구랑 같이 브런치 먹었는데 분위기도 좋고 ...


In [52]:
menupan = menupan.set_index('res_name')

In [53]:
menupan.head()

,review
res_name,
레빠쥬(les pages),파스타가 괜찮은 음식점 감사합니다 추천할께요 관자의 쫄길함이 살아있다 이곳 파스타가...
듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들 타르트가 촉...
팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 동남아음식도 좋네요 ㅎㅎ 동남아음식에 대한 거부...
더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요 맛있...
시리얼고메(Serial Gourmet),맛도 그닥 서비스는 최악이더군요 친구랑 같이 브런치 먹었는데 분위기도 좋고 맛있었어...


In [54]:
## tokenizer import
!pip install konlpy  
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Okt

     |████████████████████████████████| 19.4MB 26.3MB/s 
     |████████████████████████████████| 460kB 46.7MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# review 토큰화

In [90]:
## 토큰화 함수 : 토큰 type 변경하고 싶으면 return 부분 수정
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

def tokenizer(row):
  return hannanum.morphs(row)

In [91]:
menupan['rev_token'] = menupan['review'].apply(tokenizer)
menupan.head()

,review,rev_token
res_name,,
레빠쥬(les pages),파스타가 괜찮은 음식점 감사합니다 추천할께요 관자의 쫄길함이 살아있다 이곳 파스타가...,"[파스타, 가, 괜찮은, 음식점, 감사합니다, 추천, 할께요, 관자, 의, 쫄길함,..."
듀크렘(DEUX CREMES),직원들의 싸가지 없는 태도와 불친절을 넘어선 싸우자고 덤비는 진상직원들 타르트가 촉...,"[직원, 들, 의, 싸가지, 없는, 태도, 와, 불친절, 을, 넘어선, 싸우자고, ..."
팬아시아 신사가로수길점,보기에도 한번 가보고 싶은 곳이네요 동남아음식도 좋네요 ㅎㅎ 동남아음식에 대한 거부...,"[보기, 에도, 한번, 가보고, 싶은, 곳, 이네, 요, 동남아, 음식, 도, 좋네..."
더가든키친(The Garden Kitchen),기분좋은날 더 기분좋게 해줄 음식과 장소를 찾는다면 무조건 더가든키친인것같아요 맛있...,"[기분좋은날, 더, 기분, 좋게, 해줄, 음식, 과, 장소, 를, 찾는다면, 무조건..."
시리얼고메(Serial Gourmet),맛도 그닥 서비스는 최악이더군요 친구랑 같이 브런치 먹었는데 분위기도 좋고 맛있었어...,"[맛, 도, 그닥, 서비스, 는, 최악, 이, 더군요, 친구, 랑, 같이, 브런치,..."


In [92]:
menupan_token = menupan[['rev_token']]
menupan_token.head()

,rev_token
res_name,
레빠쥬(les pages),"[파스타, 가, 괜찮은, 음식점, 감사합니다, 추천, 할께요, 관자, 의, 쫄길함,..."
듀크렘(DEUX CREMES),"[직원, 들, 의, 싸가지, 없는, 태도, 와, 불친절, 을, 넘어선, 싸우자고, ..."
팬아시아 신사가로수길점,"[보기, 에도, 한번, 가보고, 싶은, 곳, 이네, 요, 동남아, 음식, 도, 좋네..."
더가든키친(The Garden Kitchen),"[기분좋은날, 더, 기분, 좋게, 해줄, 음식, 과, 장소, 를, 찾는다면, 무조건..."
시리얼고메(Serial Gourmet),"[맛, 도, 그닥, 서비스, 는, 최악, 이, 더군요, 친구, 랑, 같이, 브런치,..."


In [93]:
## 리뷰들 중 최대/평균 토큰 개수
print('최대 토큰 개수 :', (max(len(i) for i in menupan_token['rev_token'])))
print('평균 토큰 개수 :', sum(map(len, menupan_token['rev_token']))/len(menupan_token['rev_token']))

최대 토큰 개수 : 1759
평균 토큰 개수 : 156.39985433357612


# 단어 임베딩(word2vec)

In [94]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = menupan_token['rev_token'], size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [95]:
## embedding matrix size
model.wv.vectors.shape

(8179, 100)

In [107]:
## 키워드 입력시 유사한 단어 10개 출력
model_result = model.wv.most_similar('바삭바삭')
print(model_result)

[('실하고', 0.9648128747940063), ('바삭바삭해서', 0.9508832693099976), ('꼬들꼬들', 0.9490357637405396), ('좋았어여', 0.9447538256645203), ('나면서', 0.9425910711288452), ('촉촉해요', 0.9419444799423218), ('진한데', 0.9410488605499268), ('얇고', 0.9398005604743958), ('면도', 0.9386825561523438), ('촉촉하고', 0.9386301040649414)]
